In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive/')

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=['EIN'])

Mounted at /content/drive/


In [2]:
#Assign variables from the dataset's features, and assign the cutoff point for later binning
app_count = application_df.value_counts('APPLICATION_TYPE')
application_types_to_replace = list(app_count[app_count < 300].index)
class_count = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(class_count[class_count < 1000].index)



In [3]:
#Binning "rare" categorical variables into "Other" value
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [4]:
#Encode categorical dataset into numerical data for processing
application_converterd_to_numerical = pd.get_dummies(application_df)

application_converterd_to_numerical

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,NAME_13TH BOMB SQUADRON ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
# Split our preprocessed data into our features and target arrays
x_Data = application_converterd_to_numerical.drop(['IS_SUCCESSFUL'], axis =1)
y_Data = application_converterd_to_numerical['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(x_Data, y_Data, random_state=1, stratify= y_Data)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
#Define a process that creates a new Sequential model with hyperparameter options

number_input_features = len(X_train_scaled[0])

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh', 'sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=5), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [8]:
#Import kerastuner library and instantiate the tuner to perform the hypertuning.
!pip install keras-tuner --upgrade
import keras_tuner as kt
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=2,
    hyperband_iterations=2)

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 4 Complete [00h 00m 53s]
val_accuracy: 0.6205247640609741

Best val_accuracy So Far: 0.7928863167762756
Total elapsed time: 00h 02m 29s


In [9]:
#Print tuner results to determine the most accurate models to use on the test data
tuner.results_summary(3)

Results summary
Results in ./untitled_project
Showing 3 best trials
Objective(name="val_accuracy", direction="max")

Trial 0000 summary
Hyperparameters:
activation: tanh
first_units: 51
num_layers: 1
units_0: 36
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.7928863167762756

Trial 0003 summary
Hyperparameters:
activation: relu
first_units: 71
num_layers: 3
units_0: 31
units_1: 21
units_2: 86
units_3: 11
units_4: 66
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.6205247640609741

Trial 0001 summary
Hyperparameters:
activation: relu
first_units: 6
num_layers: 1
units_0: 96
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.6078134179115295


In [10]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 51, 'num_layers': 1, 'units_0': 36, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 71, 'num_layers': 3, 'units_0': 31, 'units_1': 21, 'units_2': 86, 'units_3': 11, 'units_4': 66, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 6, 'num_layers': 1, 'units_0': 96, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [11]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
     

268/268 - 2s - loss: 0.6085 - accuracy: 0.7929 - 2s/epoch - 6ms/step
Loss: 0.608494222164154, Accuracy: 0.7928863167762756
268/268 - 2s - loss: 0.5931 - accuracy: 0.6205 - 2s/epoch - 8ms/step
Loss: 0.5931374430656433, Accuracy: 0.6205247640609741
268/268 - 2s - loss: 0.6131 - accuracy: 0.6078 - 2s/epoch - 8ms/step
Loss: 0.6130549907684326, Accuracy: 0.6078134179115295


In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])

nn_model = tf.keras.models.Sequential()

# First hidden layer -- Changed the activation functiom from "Relu" to "tanh" (Model Optimization Method 2)
nn_model.add(tf.keras.layers.Dense(units=31, activation="tanh", input_dim=number_input_features))

# Second hidden layer -- Changed the activation functiom from "Relu" to "tanh" (Model Optimization Method 2)
nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Third hidden layer -- Changed the activation functiom from "Relu" to "tanh" (Model Optimization Method 2)
nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Fourth hidden layer -- Changed the activation functiom from "Relu" to "tanh" (Model Optimization Method 2)
nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh"))


# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()
     
     

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 31)                607972    
                                                                 
 dense_4 (Dense)             (None, 1)                 32        
                                                                 
 dense_5 (Dense)             (None, 1)                 2         
                                                                 
 dense_6 (Dense)             (None, 1)                 2         
                                                                 
 dense_7 (Dense)             (None, 1)                 2         
                                                                 
Total params: 608,010
Trainable params: 608,010
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#Reduced number of epochs from 100 to 50 for better optimization (Model Optimization Method 3)
fit_model = nn_model.fit(X_test_scaled, y_test, epochs=50) 

Epoch 1/50
268/268 [==============================] - 4s 12ms/step - loss: 0.6577 - accuracy: 0.7301
Epoch 2/50
268/268 [==============================] - 7s 25ms/step - loss: 0.5886 - accuracy: 0.8017
Epoch 3/50
268/268 [==============================] - 6s 23ms/step - loss: 0.5468 - accuracy: 0.7993
Epoch 4/50
268/268 [==============================] - 6s 22ms/step - loss: 0.5208 - accuracy: 0.8010
Epoch 5/50
268/268 [==============================] - 7s 27ms/step - loss: 0.5022 - accuracy: 0.8045
Epoch 6/50
268/268 [==============================] - 5s 20ms/step - loss: 0.4960 - accuracy: 0.8001
Epoch 7/50
268/268 [==============================] - 5s 20ms/step - loss: 0.4909 - accuracy: 0.8003
Epoch 8/50
268/268 [==============================] - 4s 13ms/step - loss: 0.4836 - accuracy: 0.8034
Epoch 9/50
268/268 [==============================] - 3s 12ms/step - loss: 0.4710 - accuracy: 0.8124
Epoch 10/50
268/268 [==============================] - 3s 11ms/step - loss: 0.4762 - accura

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.2012 - accuracy: 0.9489 - 994ms/epoch - 4ms/step
Loss: 0.20123116672039032, Accuracy: 0.948921263217926


In [15]:
# Export our model to HDF5 file
# Define the filename
filename = '/content/drive/MyDrive/AlphabetSoupCharity_Optimization2.h5'

# Save the model to a HDF5 file
nn_model.save(filename)